In [1]:
from docx import Document
import requests
import csv
import json
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
document = Document()

API_KEY = os.environ.get("API_KEY")


OFFICER_NAME = os.environ.get("OFFICER_NAME")
OFFICER_DOB = os.environ.get("OFFICER_DOB")

URL = f"https://api.company-information.service.gov.uk/search/officers?q=\"{OFFICER_NAME}\""

headers = {"Authorization": API_KEY}

print(API_KEY, OFFICER_NAME, OFFICER_DOB)

eb613ae5-33b1-4aaf-8f23-18ad86769709 Keith Ranger DOLLIVER 1970-04


In [6]:
# Add a title to the document
document.add_heading(f"Associated companies for: {OFFICER_NAME}", level=1)
current_heading = document.add_heading('Current appointments', level=1)
current_end = document.add_paragraph('-')
former_heading = document.add_heading('Former appointments', level=1)
former_end = document.add_paragraph('--')

document

In [9]:
# Load the SIC codes and descriptions from the CSV file
with open("SIC07_CH_condensed_list_en.csv", newline='') as csvfile:
    sic_codes_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    sic_mapping = {row[0]: row[1] for row in sic_codes_reader}

    sic_mapping

In [11]:
# Make the API request
response = requests.get(URL, headers=headers)
data = response.json()

data

{'items': [{'appointment_count': 1,
   'title': 'Keith Ranger DOLLIVER',
   'description_identifiers': ['appointment-count', 'born-on'],
   'kind': 'searchresults#officer',
   'address_snippet': '5580 179th Avenue Se, Bellevue, Washington 98006, Usa',
   'address': {'locality': 'Washington 98006',
    'address_line_2': 'Bellevue',
    'address_line_1': '5580 179th Avenue Se',
    'region': 'Usa'},
   'links': {'self': '/officers/hgf_hGSfHEbv9YaGnc7D8ZW6Ojo/appointments'},
   'description': 'Total number of appointments 1 - Born January 1963',
   'matches': {'snippet': [], 'title': [1, 5, 7, 12, 14, 21]},
   'date_of_birth': {'year': 1963, 'month': 1},
   'snippet': ''},
  {'matches': {'snippet': [], 'title': [1, 5, 7, 12, 14, 21]},
   'date_of_birth': {'month': 1, 'year': 1963},
   'snippet': '',
   'description': 'Total number of appointments 1 - Born January 1963',
   'address': {'premises': 'One Microsoft Way',
    'country': 'United States',
    'locality': 'Wa 98052',
    'address

In [16]:
# only keep officers with exact name match
exact_name_matches = []
for item in data['items']:
    if item['title'].lower() == OFFICER_NAME.lower():
        exact_name_matches.append(item)

exact_name_matches

[{'appointment_count': 1,
  'title': 'Keith Ranger DOLLIVER',
  'description_identifiers': ['appointment-count', 'born-on'],
  'kind': 'searchresults#officer',
  'address_snippet': '5580 179th Avenue Se, Bellevue, Washington 98006, Usa',
  'address': {'locality': 'Washington 98006',
   'address_line_2': 'Bellevue',
   'address_line_1': '5580 179th Avenue Se',
   'region': 'Usa'},
  'links': {'self': '/officers/hgf_hGSfHEbv9YaGnc7D8ZW6Ojo/appointments'},
  'description': 'Total number of appointments 1 - Born January 1963',
  'matches': {'snippet': [], 'title': [1, 5, 7, 12, 14, 21]},
  'date_of_birth': {'year': 1963, 'month': 1},
  'snippet': ''},
 {'matches': {'snippet': [], 'title': [1, 5, 7, 12, 14, 21]},
  'date_of_birth': {'month': 1, 'year': 1963},
  'snippet': '',
  'description': 'Total number of appointments 1 - Born January 1963',
  'address': {'premises': 'One Microsoft Way',
   'country': 'United States',
   'locality': 'Wa 98052',
   'address_line_1': 'Redmond'},
  'links'

In [17]:
exact_name_dob_matches = []
for item in data['items']:
    if item['title'].lower() == OFFICER_NAME.lower() and isinstance(item.get('date_of_birth'), dict) and str(item['date_of_birth']['year'])+'-'+str(item['date_of_birth']['month']).zfill(2) == OFFICER_DOB[:7]:
        exact_name_dob_matches.append(item)

exact_name_dob_matches

[]

In [18]:
data['items']

[{'appointment_count': 1,
  'title': 'Keith Ranger DOLLIVER',
  'description_identifiers': ['appointment-count', 'born-on'],
  'kind': 'searchresults#officer',
  'address_snippet': '5580 179th Avenue Se, Bellevue, Washington 98006, Usa',
  'address': {'locality': 'Washington 98006',
   'address_line_2': 'Bellevue',
   'address_line_1': '5580 179th Avenue Se',
   'region': 'Usa'},
  'links': {'self': '/officers/hgf_hGSfHEbv9YaGnc7D8ZW6Ojo/appointments'},
  'description': 'Total number of appointments 1 - Born January 1963',
  'matches': {'snippet': [], 'title': [1, 5, 7, 12, 14, 21]},
  'date_of_birth': {'year': 1963, 'month': 1},
  'snippet': ''},
 {'matches': {'snippet': [], 'title': [1, 5, 7, 12, 14, 21]},
  'date_of_birth': {'month': 1, 'year': 1963},
  'snippet': '',
  'description': 'Total number of appointments 1 - Born January 1963',
  'address': {'premises': 'One Microsoft Way',
   'country': 'United States',
   'locality': 'Wa 98052',
   'address_line_1': 'Redmond'},
  'links'